<a href="https://colab.research.google.com/github/yolomachine/RNN/blob/master/RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
!pip install markovify
!pip install pronouncing

    100% |████████████████████████████████| 942kB 18.8MB/s 
  Running setup.py bdist_wheel for pronouncing ... - done
  Stored in directory: /root/.cache/pip/wheels/81/fd/e8/fb1a226f707c7e20dbed4c43f81b819d279ffd3b0e2f06ee13
Successfully built pronouncing


In [0]:
import os
import re
import numpy as np
import pandas as pd
import warnings
import keras
import pronouncing
import markovify
import math
import random
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.utils import np_utils
from pathlib import Path
warnings.filterwarnings(action='ignore', category=DeprecationWarning)

In [0]:
class RNN:
  def __init__(self, depth, artist, max_syllables):
    self.artist = artist
    self.training = False
    self.max_syllables = max_syllables
    self.model = Sequential()
    self.model.add(LSTM(4, input_shape=(2, 2), return_sequences=True))
    for i in range(depth):
      self.model.add(LSTM(8, return_sequences=True))
    self.model.add(LSTM(2, return_sequences=True))
    self.model.summary()
    self.model.compile(optimizer='rmsprop', loss='mse')
	  
  def markov(self, text):
    read = open(text, "r", encoding='utf-8').read()
    model = markovify.NewlineText(read)
    return model
  
  def syllables(self, line):
    count = 0
    for word in line.split(" "):
      vowels = 'aeiouyаеиоуэюя'
      word = word.lower().strip(".:;?!")
      if len(word) == 0:
        return 0
      if word[0] in vowels:
        count +=1
      for index in range(1,len(word)):
        if word[index] in vowels and word[index-1] not in vowels:
          count +=1
      if word.endswith('e'):
        count -= 1
      if word.endswith('le'):
        count += 1
      if count == 0:
        count += 1
    return count / self.max_syllables
  
  def rhymeindex(self, lyrics):
    if self.artist + ".rhymes" in os.listdir('gdrive/My Drive') and self.training == False:
      print("Loading saved rhymes: " + self.artist + ".rhymes")
      return open('gdrive/My Drive/' + self.artist + ".rhymes", "r",encoding='utf-8').read().split("\n")
    else:
      rhyme_master_list = []
      for i in lyrics:
        word = re.sub(r"\W+", '', i.split(" ")[-1]).lower()
        rhymeslist = pronouncing.rhymes(word)
        rhymeslistends = []      
        for i in rhymeslist:
          rhymeslistends.append(i[-2:])
        try:
          rhymescheme = max(set(rhymeslistends), key=rhymeslistends.count)
        except Exception:
          rhymescheme = word[-2:]
        rhyme_master_list.append(rhymescheme)
      rhyme_master_list = list(set(rhyme_master_list))
      reverselist = [x[::-1] for x in rhyme_master_list]
      reverselist = sorted(reverselist)
      rhymelist = [x[::-1] for x in reverselist]
      print("List of Sorted 2-Letter Rhyme Ends:")
      print(rhymelist)
      f = open('gdrive/My Drive/' + self.artist + ".rhymes", "w", encoding='utf-8')
      f.write("\n".join(rhymelist))
      f.close()
      return rhymelist
    
  def rhyme(self, line, rhyme_list):
    word = re.sub(r"\W+", '', line.split(" ")[-1]).lower()
    rhymeslist = pronouncing.rhymes(word)
    rhymeslistends = []
    for i in rhymeslist:
      rhymeslistends.append(i[-2:])
    try:
      rhymescheme = max(set(rhymeslistends), key=rhymeslistends.count)
    except Exception:
      rhymescheme = word[-2:]
    try:
      float_rhyme = rhyme_list.index(rhymescheme)
      float_rhyme = float_rhyme / float(len(rhyme_list))
      return float_rhyme
    except Exception:
      float_rhyme = None
      return float_rhyme
    
  def split_lyrics(self, file):
    text = open(file, encoding='utf-8').read().split("\n")
    while "" in text:
      text.remove("")
    return text
  
  def generate_lyrics(self, file):
    bars = []
    last_words = []
    lyriclength = len(open(file,encoding='utf-8').read().split("\n"))
    print("Length to generate: " + str(lyriclength))
    count = 0
    markov_model = self.markov(file)

    times = 0
    while len(bars) < lyriclength / 9 and count < lyriclength * 2:
      times += 1
      if times > 50000:
        break
      c = len(bars)
      bar = markov_model.make_sentence(tries=100)
      if type(bar) != type(None):
        def get_last_word(bar):
          last_word = bar.split(" ")[-1]
          if last_word[-1] in "!.?,":
            last_word = last_word[:-1]
          return last_word
        last_word = get_last_word(bar)
        if last_words.count(last_word) < 2:
          bars.append(bar)
          last_words.append(last_word)
          count += 1
      if (len(bars) > c):
        print(bars)
    return bars
  
  def build_dataset(self, lines, rhyme_list):
    dataset = []
    line_list = []
    for line in lines:
      line_list = [line, self.syllables(line), self.rhyme(line, rhyme_list)]
      dataset.append(line_list)
    x_data = []
    y_data = []
    for i in range(len(dataset) - 3):
      line1 = dataset[i    ][1:]
      line2 = dataset[i + 1][1:]
      line3 = dataset[i + 2][1:]
      line4 = dataset[i + 3][1:]
      x = [line1[0], line1[1], line2[0], line2[1]]
      x = np.array(x)
      x = x.reshape(2,2)
      x_data.append(x)
      y = [line3[0], line3[1], line4[0], line4[1]]
      y = np.array(y)
      y = y.reshape(2,2)
      y_data.append(y)
    x_data = np.array(x_data)
    y_data = np.array(y_data)
    return x_data, y_data
  
  def compose_vectors(self, lines, rhyme_list, lyrics_file):
    poem_vectors = []
    human_lyrics = self.split_lyrics(lyrics_file)
    initial_index = random.choice(range(len(human_lyrics) - 1))
    initial_lines = human_lyrics[initial_index:initial_index + 2]
    starting_input = []
    for line in initial_lines:
      starting_input.append([self.syllables(line), self.rhyme(line, rhyme_list)])
    starting_vectors = self.model.predict(np.array([starting_input]).flatten().reshape(1, 2, 2))
    poem_vectors.append(starting_vectors)
    for i in range(100):
      poem_vectors.append(self.model.predict(np.array([poem_vectors[-1]]).flatten().reshape(1, 2, 2)))
    return poem_vectors
    
  def vectors_into_poem(self, vectors, generated_lyrics, rhyme_list):

    def last_word_compare(poem, line2):
      penalty = 0 
      for line1 in poem:
        word1 = line1.split(" ")[-1]
        word2 = line2.split(" ")[-1]
        while word1[-1] in "?!,. ":
          word1 = word1[:-1]
        while word2[-1] in "?!,. ":
          word2 = word2[:-1]
        if word1 == word2:
          penalty += 0.2
      return penalty

    def calculate_score(vector_half, syllables, rhyme, penalty):
      desired_syllables = vector_half[0]
      desired_rhyme = vector_half[1]
      desired_syllables = desired_syllables * self.max_syllables
      desired_rhyme = desired_rhyme * len(rhyme_list)
      score = 1.0 - abs(float(desired_syllables) - float(syllables)) + abs(float(desired_rhyme) - float(rhyme)) - penalty
      return score
    
    dataset = []
    for line in generated_lyrics:
      line_list = [line, self.syllables(line), self.rhyme(line, rhyme_list)]
      dataset.append(line_list)
    poem = []
    vector_halves = []
    for vector in vectors:
      vector_halves.append(list(vector[0][0])) 
      vector_halves.append(list(vector[0][1]))
    for vector in vector_halves:
      scorelist = []
      for item in dataset:
        line = item[0]
        if len(poem) != 0:
          penalty = last_word_compare(poem, line)
        else:
          penalty = 0
        total_score = calculate_score(vector, item[1], item[2], penalty)
        score_entry = [line, total_score]
        scorelist.append(score_entry)
      fixed_score_list = [0]
      for score in scorelist:
        fixed_score_list.append(float(score[1]))
      max_score = max(fixed_score_list)
      for item in scorelist:
        if item[1] == max_score:
          poem.append(item[0])
          print (str(item[0]))
          for i in dataset:
            if item[0] == i[0]:
              dataset.remove(i)
              break
          break     
    return poem
    
  def train(self, x_data, y_data):
    self.model.fit(
        np.array(x_data), 
        np.array(y_data),
        batch_size=2,
        epochs=5,
        verbose=1
    )
    self.save(self.artist)
    
  def process(self, file, output):
    if self.training == True:
      bars = self.split_lyrics(file)
    if self.training == False:
      self.load(self.artist)
      print("Preparing to generate: " + output)
      bars = self.generate_lyrics(file)
    rhyme_list = self.rhymeindex(bars)
    
    if self.training == True:
      x_data, y_data = self.build_dataset(bars, rhyme_list)
      self.train(x_data, y_data)
    if self.training == False:
      print("Composing poem")
      vectors = self.compose_vectors(bars, rhyme_list, file)
      poem = self.vectors_into_poem(vectors, bars, rhyme_list)
      f = open(output, "w", encoding='utf-8')
      for bar in poem:
        f.write(bar)
        f.write("\n")
    
  def save(self, artist):
    print("Saving network: " + self.artist +".hdf5")
    self.model.save_weights('gdrive/My Drive/' + artist + '.hdf5')
    print("Saved network: " + self.artist +".hdf5")
    
  def load(self, artist):
    if artist + '.hdf5' in os.listdir('gdrive/My Drive'):
      print("Loading saved network: " + artist + ".hdf5")
      self.model.load_weights('gdrive/My Drive/' + artist + '.hdf5')
      print("Loaded saved network: " + artist + ".hdf5")

  

In [23]:
depth = 4 
max_syllables = 8
artist = "gomeniuk.aa2"
composition = "gdrive/My Drive/composition2.txt"
sample = "gdrive/My Drive/pushkin2.txt"

network = RNN(depth, artist, max_syllables)
network.training = True
network.process(sample, composition)

network = RNN(depth, artist, max_syllables)
network.training = False
network.process(sample, composition)

print("Done")

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_91 (LSTM)               (None, 2, 4)              112       
_________________________________________________________________
lstm_92 (LSTM)               (None, 2, 8)              416       
_________________________________________________________________
lstm_93 (LSTM)               (None, 2, 8)              544       
_________________________________________________________________
lstm_94 (LSTM)               (None, 2, 8)              544       
_________________________________________________________________
lstm_95 (LSTM)               (None, 2, 8)              544       
_________________________________________________________________
lstm_96 (LSTM)               (None, 2, 2)              88        
Total params: 2,248
Trainable params: 2,248
Non-trainable params: 0
_________________________________________________________________
List of 

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Не буду увлечен! В бездействии ночном живей горят во мне болезни жар враждебный.
Не буду увлечен! В бездействии ночном живей горят во мне расстроенным почли.
Не буду увлечен! В бездействии ночном живей горят во мне болезни жар враждебный.
Барков шутливых од тебе не смею над тобой забавилась природа,
Я утром наконец являлся к милой деве даст название супруги.
Счастливы юноши! Но кто, скажи, меж ими сестра их Елица.
И, вдохновенный, нарек младенца Феоном. За чашей медленной афею иль деисту,
О, как мучительно тобою счастлив я, когда я был у берегов,
А теперь мне жизнь – ряд горестей, гремяща слава – отрада;
Одни, без ангелов, под пальмою видит он отца и брата.
У Граций в отпуску, и у любви в душе моей читать? Ах, и самой
В трудах, заботах и в прозе, и в ответ получил следующее письмо:
Познал и тихой труд, и собственность, и время земледельца.
Давно повешенных и там из вас на радость соседей-врагов,
Волшебнице-Мечте, шепнувшей: ты поэт, несметные богатства,
Врата отверсты в тьме ночной их 

In [0]:
with open("gdrive/My Drive/pushkin.txt") as f:
    text = f.read()

# Build the model.
text_model = markovify.Text(text)

# Print five randomly-generated sentences
for i in range(10):
    print(text_model.make_sentence())

# Print three randomly-generated sentences of no more than 140 characters
for i in range(30):
    print(text_model.make_short_sentence(100, tries=100))